In [ ]:
# This schedular performs sample transfers for small files. 
# then it performs gets the local maxima from the throughput curve.

# Pre-requisite:

# Libraries:

In [26]:
import os
import random
import sys, time
import datetime
import math

# Variables:


In [27]:
# Feel free to update those variable. If you know 
# what you are doing! :-)

# node information: 
source = 'evenstar.cse.buffalo.edu'
destination = 'didclab-ws10.cse.buffalo.edu'
port = 50000
source_path = "/home/zulkar/dataset/first_one/"
destination_path = "/home/zulkar/received/"


# Network information: 
bandwidth = 1  # Gbps
rtt = 0.2   # ms

# Dataset information:
# group range is a dictionary with key as group names:
# group means file size range e.g. small, medium, large 
# value of this dictionary is a list:
#      first element : lowest filesize
#      second element : hight filesize
#      third element : number of files
#      fourth element : sample folders
group_range = {'1':[1,40,100,10],'2':[100,600,50,4],'3':[900,10000,10,10]}
ind_low  = 0
ind_high = 1
ind_n = 2
ind_samples = 3


#################### Derived variables : Don't Set those ! ################
full_source = "file://" + source_path  + "/"
full_destination = 'ftp://' + destination + ":" + str(port) + destination_path

buffer_size = int((bandwidth*1024*rtt*1000)/8)


######################################################################

print("source node: ", source)
print("destination node: ", destination)
print("control port: ", port, " This port is same to both source and destination, just to make life easy")
print("dataset path in source: ", source_path)
print("destination path: ", destination_path)
print("link bandwidth: ", bandwidth)
print("rtt: ", rtt)
print("buffer size: ", buffer_size)

source node:  evenstar.cse.buffalo.edu
destination node:  didclab-ws10.cse.buffalo.edu
control port:  50000  This port is same to both source and destination, just to make life easy
dataset path in source:  /home/zulkar/dataset/first_one/
destination path:  /home/zulkar/received/
link bandwidth:  1
rtt:  0.2
buffer size:  25600


# Methods:

In [28]:
def getFolderSize(folder):
    """ This function takes full path of a folder as input
        Returns size of whole folder in (bytes)
    """
    total_size = os.path.getsize(folder)
    for item in os.listdir(folder):
        itempath = os.path.join(folder, item)
        if os.path.isfile(itempath):
            total_size += os.path.getsize(itempath)
        elif os.path.isdir(itempath):
            total_size += getFolderSize(itempath)
    return total_size # bytes

In [29]:
def perform_transfer(source, destination, p = 1, cc=1, pp=1, fast=True, tcp_bs=25600):
    """To do a transfer you need to provide:
            source 
            destination
            p
            cc
            pp
            fast
            tcp-bs
       performs the transfer and returns 
           throughput
           time
    """
    space = " "
    globus_cmd = "globus-url-copy" + \
                    space + "-tcp-bs" + space + str(tcp_bs) + \
                    space + "-p" + space + str(p) + \
                    space + "-cc" + space + str(cc) + \
                    space + "-pp" + space + str(pp) + \
                    space + full_source + \
                    space + full_destination
    globus_cmd 
    
    print("started data transfer: ")
    print("globus command:",globus_cmd)
    
    
    transfer_start_time = time.time()
    os.system(globus_cmd)
    transfer_finish_time = time.time()  # in seconds
    
    time_required = transfer_finish_time - transfer_start_time 
    bytes_xfered = getFolderSize(sample_path)
    
    
    throughput = ( bytes_xfered *8) / (time_required * 1024 * 1024 )     # in Giga bit per seconds
    throughput
    
    print("transfer finished !")
    print("bytes transferred: ", bytes_xfered)
    print("time needed for transfer: ", time_required, "s.")
    print("throughput: ", throughput)
    
    return time_required, throughput

# Schedular:

In [30]:
# perform initial transfer from small files.
# group_1 is the small files:


In [35]:
# initialize sample data structure for samples to be transferred.
# its a three level deep dictionary:
# level one : map - group_id to sample_id
# level two : map - sample_id to sample_fields
# level three : map sample_fields to corresponding values
# sample_info = { <group_id> : { <sample_id> : { 'path': <path_val>,
#                                         'size': <samplesize in bytes>,
#                                         'transferred': <transferred_or_not boolean>,
#                                         'throughput' : <throughput_val in Gbps>,
#                                         'transfer-time' : <val>,
#                                         'p' : <p_value>,
#                                         'cc' : <cc_value>,
#                                         'pp' : <pp_value>,
#                                         'fast' : <fast value>,
#                                         'tcp-bs': <tcp_buffer size>,
#                                         'traffic-intensity': <intensity from 0 to 1>}}}

sample_info = {}
for key, value in group_range.items():
    group_id = key
    n_samples = value[ind_samples]
    
    level_two_dict = {}
    for sample in range(n_samples):
        level_three_dict = {}
        
        # Compute path:
        sample_folder_name = 'sample_' + group_id + '_' + str(sample)
        full_sample_path = source_path + sample_folder_name
        
        level_three_dict['path'] = full_sample_path
        
        # Compute size:
        level_three_dict['size'] = getFolderSize(full_sample_path)
        
        level_three_dict['transferred'] = False
        
        level_three_dict['p'] = 0
        
        level_three_dict['cc'] = 0
        
        level_three_dict['pp'] = 0
        
        level_three_dict['fast'] = 'false'
        
        level_three_dict['tcp-bs'] = 25600
        
        level_three_dict['throughput'] = 0
        
        level_three_dict['transfer-time'] = 0 
        
        
        # Add level three dict to level two dict
        sample_name = str(sample)
        level_two_dict[sample_name] = level_three_dict
    
    # Add level two dict to level one dict  (sample_info)  
    sample_info[group_id] = level_two_dict
#sample_info

In [36]:
sample_info['1']['2']

{'cc': 0,
 'fast': 'false',
 'p': 0,
 'path': '/home/zulkar/dataset/first_one/sample_1_2',
 'pp': 0,
 'size': 80744448,
 'tcp-bs': 25600,
 'throughput': 0,
 'transfer-time': 0,
 'transferred': False}

In [15]:
# Get the total dataset size in bytes:
dataset_size = getFolderSize(source_path)
print("Total size of dataset :", dataset_size/(1024*1024), "Mbytes")

Total size of dataset : 146766.29296875 Mbytes


In [19]:
# get information of all small samples:
group_id = '1'
sample_id = '0'

# get local 



10